In [6]:
import torch
import torch.nn as nn
import os
import re
from pathlib import Path

# import gluonnlp
import numpy as np
from scipy import stats
import torch
import torch.nn as nn 

GPUIdx = str("0")

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= GPUIdx

from tqdm.notebook import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [7]:
### Utils
# import gluonnlp
import copy
import random
# import gensim.downloader

crit_mean = nn.MSELoss()
crit_sum = nn.MSELoss(reduction='sum')
cos = nn.CosineSimilarity(eps=1e-6)

### Data Loading
wordsim_sim = []
file_object = open("./wordsim_data/wordsim353/wordsim353_sim_rel/wordsim_similarity_goldstandard.txt", 'r', encoding='utf-8')
for line in file_object:
    line = line.split()
    wordsim_sim.append([line[0], line[1], float(line[2])])
file_object.close()
            
wordsim_rel = []
file_object = open("./wordsim_data/wordsim353/wordsim353_sim_rel/wordsim_relatedness_goldstandard.txt", 'r', encoding='utf-8')
for line in file_object:
    line = line.split()
    wordsim_rel.append([line[0], line[1], float(line[2])])
file_object.close()

rw = []
file_object = open("./wordsim_data/rw/rw.txt", 'r', encoding='utf-8')
for line in file_object:
    line = line.split()
    rw.append([line[0], line[1], float(line[2])])
file_object.close()
            
men = []
file_object = open("./wordsim_data/men/MEN/MEN_dataset_natural_form_full", 'r', encoding='utf-8')
for line in file_object:
    line = line.split()
    men.append([line[0], line[1], float(line[2])])
file_object.close()
            
sem = []
file_object1 = open("./wordsim_data/SemEval17-Task2/test/subtask1-monolingual/data/en.test.data.txt", 'r', encoding='utf-8')
file_object2 = open("./wordsim_data/SemEval17-Task2/test/subtask1-monolingual/keys/en.test.gold.txt", 'r', encoding='utf-8')
for line1, line2 in zip(file_object1, file_object2):
    line1 = line1.split()
    line2 = line2.split()
    sem.append([line1[0], line1[1], float(line2[0])])
file_object1.close()
file_object2.close()
        
simlex = []
file_object = open("./wordsim_data/SimLex-999/SimLex-999.txt", 'r', encoding='utf-8')
file_object.readline() # skip the first line
for line in file_object:
    line = line.split()
    simlex.append([line[0], line[1], float(line[3])])
file_object.close()
            
simverb = []
file_object = open("./wordsim_data/SimVerb-3000-test.txt", 'r', encoding='utf-8')
for line in file_object:
    line = line.split()
    simverb.append([line[0], line[1], float(line[3])])
file_object.close()

def read_word_vecs(filename):
    print("Vectors read from", filename)
    wordVectors = {}
    if filename.endswith('.gz'): fileObject = gzip.open(filename, 'r')
    elif filename.endswith('.h5'):
        fileObject_ = h5py.File(filename, 'r')
        fileObject = fileObject_[list(fileObject_.keys())[0]]
        wordlist = np.array(fileObject[u'axis1'])
        pbar = tqdm_notebook(total = len(wordlist))
        unk_counter = 0
        for i, w in enumerate(wordlist):
            pbar.update(1)
            wordVectors[w[6:]] = fileObject[u'block0_values'][i]
        pbar.close()
    else:
        fileObject = open(filename, 'r')
        fileObject.readline() # For handling First Line
        for line in fileObject:
            line = line.strip()
            word = line.split()[0]
            wordVectors[word] = np.zeros(len(line.split())-1, dtype=np.float64)
            vector = line.split()[1:]
            if len(vector) == 300:
                for index, vecVal in enumerate(vector):
                    wordVectors[word][index] = float(vecVal)
                if False:
                    wordVectors[word] = wordVectors[word] / math.sqrt((wordVectors[word]**2).sum() + 1e-5)
            else:
                print(line)
                continue
    return wordVectors

### Evaluator
def Evaluation(model_name, tokenizer, data):
    model_score = []; human_score = []
    model_name.eval()
    for d in data:
        w1, w2, score = d
#         print(w1, w2, score, end=' ')
        w1, w2 = w1.lower().strip(), w2.lower().strip()
        w1 = tokenizer(w1, return_tensors="pt", add_special_tokens=True)
        w2 = tokenizer(w2, return_tensors="pt", add_special_tokens=True)
        w1 = { k: v.to(device) for k, v in w1.items() }
        w2 = { k: v.to(device) for k, v in w2.items() }
        
        with torch.no_grad():
            w1 = model_name(**w1)["last_hidden_state"][:,0,:]
            w2 = model_name(**w2)["last_hidden_state"][:,0,:]
        model_score.append(cos(w1,w2).to('cpu').item())
        human_score.append(score)
    return round(stats.spearmanr(model_score, human_score)[0], 4)

def StaticEvaluation(model_name, data):
    model_score = []; human_score = []
    for d in data:
        w1, w2, score = d
        w1 = w1.lower()
        w2 = w2.lower()
        if w1 in model_name:
            w1 = torch.tensor(model_name[w1]).unsqueeze(0)
        else:
            w1 = torch.rand(1, 300)
        if w2 in model_name:
            w2 = torch.tensor(model_name[w2]).unsqueeze(0)
        else:
            w2 = torch.rand(1, 300)
        model_score.append(cos(w1,w2).to('cpu').item())
        human_score.append(score)
    return round(stats.spearmanr(model_score, human_score)[0], 4)

def ConcatEvaluation(model_names, data):
    model_score = []; human_score = []
    for d in data:
        w1, w2, score = d
        w1 = torch.tensor([Tokenizer.encode(w1)])#.to(device)
        w2 = torch.tensor([Tokenizer.encode(w2)])#.to(device)
        with torch.no_grad():
            w1 = torch.cat((model_names[0](w1)[0][:,0,:], model_names[1](w1)[0][:,0,:]), 1)
            w2 = torch.cat((model_names[0](w2)[0][:,0,:], model_names[1](w2)[0][:,0,:]), 1)
        model_score.append(cos(w1,w2).to('cpu').item())
        human_score.append(score)
    return round(stats.spearmanr(model_score, human_score)[0], 4)

In [8]:
from transformers import AutoTokenizer, AutoModel

# ModelName = 'bert-base-uncased'
# ModelName = 'bert-large-uncased'
# ModelName = 'roberta-base'
# ModelName = 'sentence-transformers/bert-base-nli-stsb-mean-tokens'
# ModelName = '/ckpt/def-bert/GlossBERT/'
# ModelName = '/ckpt/def-bert/save/mlm/bert/checkpoint-21605/'
# ModelName = '/ckpt/def-bert/save/W[CLS]-D[CLS]/'
# ModelName = '/ckpt/def-bert/save/D[CLS]-E[TGT]/'
# ModelName = '/ckpt/def-bert/save/W[CLS]-W[TGT]/'
# ModelName = '/ckpt/def-bert/save/W[CLS]-E[TGT]/'
# ModelName = '/ckpt/def-bert/save/W[CLS]-W[TGT]+W[CLS]-D[CLS]/'
# ModelName = '/ckpt/def-bert/save/W[CLS]-E[TGT]+W[CLS]-D[CLS]/'
# ModelName = '/ckpt/def-bert/save/W[CLS]-W[TGT]+D[CLS]-E[TGT]/'
# ModelName = '/ckpt/def-bert/save/W[CLS]-E[TGT]+D[CLS]-E[TGT]/'
# ModelName = './save/WD+DE+WW+WE_best/'
ModelName = './save/WE+WD+DE+WW/'

Tokenizer = AutoTokenizer.from_pretrained(ModelName)
Embedding = AutoModel.from_pretrained(ModelName)

Embedding = Embedding.to(device)
WS_simScore = Evaluation(Embedding, Tokenizer, wordsim_sim)
WS_relScore = Evaluation(Embedding, Tokenizer, wordsim_rel)
RWScore = Evaluation(Embedding, Tokenizer, rw)
MENScore = Evaluation(Embedding, Tokenizer, men)
SEMScore = Evaluation(Embedding, Tokenizer, sem)
SLScore = Evaluation(Embedding, Tokenizer, simlex)
SVScore = Evaluation(Embedding, Tokenizer, simverb)

# WS_simScore = StaticEvaluation(Embedding, wordsim_sim)
# WS_relScore = StaticEvaluation(Embedding, wordsim_rel)
# RWScore = StaticEvaluation(Embedding, rw)
# MENScore = StaticEvaluation(Embedding, men)
# SEMScore = StaticEvaluation(Embedding, sem)
# SLScore = StaticEvaluation(Embedding, simlex)
# SVScore = StaticEvaluation(Embedding, simverb)

print('WordSim353(Sim) Score', WS_simScore)
print('WordSim353(Rel) Score', WS_relScore)
print('RareWords Score', RWScore)
print('MEN(Test) Score', MENScore)
print('SEM(Test) Score', SEMScore)
print('SimLex Score', SLScore)
print('SimVerb Score', SVScore)
print("="*20)
print("Avg.", (WS_simScore+WS_relScore+RWScore+MENScore+SEMScore+\
               SLScore+SVScore)/7)
print("="*20)


WordSim353(Sim) Score 0.7158
WordSim353(Rel) Score 0.5179
RareWords Score 0.4674
MEN(Test) Score 0.7648
SEM(Test) Score 0.5868
SimLex Score 0.5324
SimVerb Score 0.4109
Avg. 0.5708571428571428
